### Imports

In [1]:
import os
import spotipy
import spotipy.util as util
import pandas as pd
import itertools
import json
from pprint import pprint
from pipetools import pipe

### Authentication

In [2]:
data_file = open('auth_json.json')
auth_json = json.load(data_file)

username = auth_json['username']
scope = auth_json['scope']
client_id = auth_json['client_id']
client_secret = auth_json['client_secret']
redirect_uri = auth_json['redirect_uri']
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
sp = spotipy.Spotify(auth=token)

### Helper Functions

In [3]:
def remove_key(d, key):
    r = dict(d)
    del r[key]
    return r

def merge_two_dicts(x, y):
    """Given two dicts, merge them into a new dict as a shallow copy."""
    z = x.copy()
    z.update(y)
    return z

def rename_dict_key(x, y):
    def f(d): 
        d[y] = d.pop(x)
        return d
    return f

def dict_filter(cols, pipe):
    def _filter(x):
        key_filter = {k: x[k] for k in cols}
        filtered = pipe(key_filter)
        return filtered   
    return _filter

def replace_value(d, key_to_find, definition):
    for key in d.keys():
        if key == key_to_find:
            d[key] = definition
    return d

def list_to_str(d, key, sep):
    str_genres = sep.join(d[key])
    return replace_value(d, key, str_genres)

def toDF(df): return pd.DataFrame(df, index=[0])

### Dictionary Renaming Pipelines

In [4]:
def r_name(x): return rename_dict_key("name", x+"_name")
def r_id(x): return rename_dict_key("id", x+"_id")
def r_uri(x): return rename_dict_key("uri", x+"_uri")
def r_type(x): return rename_dict_key("type", x+"_type")
def r_pop(x): return rename_dict_key("popularity", x+"_popularity")
def r_href(x): return rename_dict_key("href", x+"_href")
def uri_r_pop(x): return rename_dict_key("popularity", x+"_popularity")
def uri_r_genres(x): return rename_dict_key("genres", x+"_genres")

def parent_r_pipe(x): return pipe | r_name(x) | r_uri(x) | r_type(x) | r_pop(x) | r_href(x)
def child_r_pipe(x): return pipe | r_name(x) | r_id(x) | r_uri(x) | r_type(x) | r_href(x)
def uri_r_pipe(x): return pipe | uri_r_pop(x) | uri_r_genres(x)

def uri_rename(d, d_type, cols):
    d_uri = sp
    if d_type == "artist": 
        d_uri = sp.artist(d[d_type+'_uri']) 
    else: d_uri = sp.album(d[d_type+'_uri'])
    d_xs = {k: d_uri[k] for k in cols}
    d = merge_two_dicts(d, d_xs)
    return uri_r_pipe(d_type)(d)

### Spotify Web API Call Processor

In [5]:
tracks_keys = ['name', 'explicit', 'popularity', 'disc_number', 'uri', 'type', 'duration_ms', 'href']
artists_keys = ['name', 'type', 'id', 'uri', 'href']
albums_keys = ['name', 'type', 'id', 'uri','album_type','href']

def process_dict(search, tracks_keys, artists_keys, albums_keys):
    result = pd.DataFrame()
    if search["tracks"]['total'] != 0:
        d = search["tracks"]["items"]
        f = dict_filter(tracks_keys, parent_r_pipe("track"))
        g = dict_filter(artists_keys, child_r_pipe("artist"))
        z = dict_filter(albums_keys, child_r_pipe("album"))

        tracks_d = [f(x) for x in d]
        artists_d = [g(x["artists"][0]) for x in d]
        artists_d = [uri_rename(x, "artist", ['popularity', 'genres']) for x in artists_d]
        artists_d = [list_to_str(x, "artist_genres", ", ") for x in artists_d]
        albums_d = [z(x["album"]) for x in d]
        albums_d = [uri_rename(x, "album", ['popularity', 'genres']) for x in albums_d]
        albums_d = [list_to_str(x, "album_genres", ", ") for x in albums_d]

        tracks_dfs = [toDF(x) for x in tracks_d]
        artists_dfs = [toDF(x) for x in artists_d]
        albums_dfs = [toDF(x) for x in albums_d]

        tracks_df = pd.concat(tracks_dfs)
        artists_df = pd.concat(artists_dfs)
        albums_df = pd.concat(albums_dfs)
        df = pd.concat([tracks_df, artists_df, albums_df], axis=1)
        result = df[['track_uri', 'album_uri', 'artist_uri',  
                    'track_name', 'album_name', 'artist_name', 
                    'album_genres', 'artist_genres', 'duration_ms', 'explicit', 
                    'track_popularity', 'artist_popularity', 'album_popularity',
                    'track_type', 'album_type', 'artist_type']]
    else: result
    return result

def process(search): 
    return process_dict(search, tracks_keys, artists_keys, albums_keys)

### Text Parser

In [6]:
flatten = lambda l: [item for sublist in l for item in sublist]

def word_count(phrase, max_words):
    phrase_count = len(phrase.split(" "))
    res = ""
    if phrase_count != max_words: res = "!="+str(max_words)
    else: res = phrase
    return res

def get_phrases(text):
    words = text.split()
    ns = range(1, len(words)) # n = 1...(n-1)
    phrases = []
    for n in ns: # split into 2, 3, 4, ..., n parts.
        for idxs in itertools.combinations(ns, n):
            phrases += [
                [ x for x in [
                    word_count(phrase, 3) for phrase in [
                        ' '.join(words[i:j]) for i, j in zip((0,) + idxs, idxs + (None,))
                    ]
                ] if "!=3" not in x ]
            ]      
    return list(set(flatten(phrases)))

### Main Method: text_to_playlist

In [7]:
def text_to_playlist(text, playlist_name="Text2Playlist"):
    phrases = get_phrases(text)
    print("Phrases from '"+text+"':")
    for phrase in phrases: print(phrase)
    
    searches = []
    while True:
         try:
             searches = [sp.search(phrase) for phrase in phrases]
             break
         except SpotifyException:
             pass
    result = pd.concat([process(search) for search in searches]
                   ).drop_duplicates(subset=['track_uri'])
    
    playlist = sp.user_playlist_create(username, playlist_name, public=True)
    playlist_id = playlist['id']
    tracks = result['track_uri']
    snapshot = sp.user_playlist_add_tracks(username, playlist_id, tracks)
    playlist_url = sp.user_playlists(username)["items"][0]['external_urls']['spotify']
    print("\n")
    print("Playlist '"+playlist_name+"' URL: "+playlist_url)

### Validation

In [8]:
poem = "if i can't let it go out of my mind"    
text_to_playlist(poem)

Phrases from 'if i can't let it go out of my mind':
go out of
can't let it
of my mind
if i can't
i can't let
out of my
let it go
it go out


Playlist 'Text2Playlist' URL: http://open.spotify.com/user/joshej07/playlist/7kDXe7RglYSNoVhiKdOMzV
